# Download Vivid Mosaic Seamlines for Vivid Mosaic

In this example we are going to download the seamlines for an entire Vivid Product. We will be using a Basemaps connect ID to get access to Vivid mosaics.

#### Initialize the Maxar-OGC Module

In [1]:
Maxar_OGC import Interface

try:
  sw_ogc = Interface() # if .ogc-config was created
except:
  sw_ogc = Interface('https://securewatch.maxar.com/','<connect_id>') # if .ogc-config was not created

#### Search for available products over an area. 
This section searches for all the features over a given BBOX, then prints a list of all the products available in the area.

In [2]:
typename = "MaxarCatalogMosaicSeamlines"
bbox = "39.622615,-105.174866,39.909736,-104.713440" #BBOX is Denver, CO.

#search for products
search_results = sw_ogc.search(bbox=bbox, typename=typename, shapefile=False)

#Maxar Catalog Mosaic Seamlines searches will return the first 1000 features. If your BBOX is too big, you may not get a
#complete list of all products available. If the length is under 1000, then your BBOX is appropriately sized.
print(len(search_results))

#create a set of all the product IDs available
products = set()

for i in range(len(search_results)):
    products.add(search_results[i]['properties']['product_id'])

print(products)

87
{'VIVID2_NA25_19Q4', 'VIVID2_NA25_19Q1', 'VIVID_NA25_20Q2', 'VIVID_NA25_20Q4', 'BBA_BGRN_VIVID_NA25_20Q4', 'Vivid_Standard_US20_21Q4', 'Vivid_Standard_30_US20_21Q4', 'Vivid_Basic_US20_21Q4', 'BBA_BGRN_VIVID_NA25_20Q2'}


#### Decide which product you want to download from the above list. In this example we are using 'Vivid_Standard_US20_21Q4'.
If the product has less than 1000 features, you can download the whole thing in one request using the following code.

In [3]:
#create filter based on product ID.
product_id = 'Vivid_Standard_US20_21Q4'
filter = "(product_id = '{}')".format(product_id)

#Note that some products may contain over 1000 features. In this case, the downloaded SHP will have exactly 1000 
#features because that's where the download will stop. When downloading these features, ensure you provide a BBOX argument that
#reduces the amount of features to <1000.
search_results = sw_ogc.search(typename=typename, shapefile=False, filter=filter)
print(len(search_results))

#Declare download path
download_path = "C:\\Users\\wi008132\\Downloads\\example.zip"

#Download it by adding the argument Shapefile = True and dropping the BBOX.
sw_ogc.search(typename=typename, shapefile=True, filter=filter, download_path=download_path)

926


'C:\\Users\\wi008132\\Downloads\\example_dup_dup.zip'

#### If the product you choose has over 1000 features, you'll need to split the requests into multiple BBOXs to download all the seamlines. In this example, we'll use product_id = 'BBA_BGRN_VIVID_NA25_20Q4'
The following code uses the MaxarCatalogMosaicProducts typename to extract the entire extent of the product.

In [4]:
typename = "MaxarCatalogMosaicProducts"

#This product ID has over 1000 features
product_id = 'BBA_BGRN_VIVID_NA25_20Q4'
filter = "(product_id = '{}')".format(product_id)

#search for the chosen mosaic product
search_results = sw_ogc.search(typename=typename, shapefile=False, filter = filter)

#extract the polygon of the product
coords = search_results[0]['geometry']['coordinates']

#takes the polygon and converts it to a BBOX of the extent
def findExtent(coords):
    lats = []
    lons = []

    for XYpair in coords[0][0]:
        lons.append(XYpair[0])
        lats.append(XYpair[1])

    extent = (min(lats), min(lons), max(lats), max(lons))
    return(extent)

#Run the function to find the extent of the product
extentBBOX = findExtent(coords)
print(extentBBOX)

(36.4746, -113.291, 49.3066, -100.459)


#### Take the extent and break it into multiple BBOXs

In [5]:
#This function takes the extent of the product and breaks the into smaller BBOX's.
def BBOX_creation(extentBBOX):
    #Define variables from the extent
    MinY = extentBBOX[0]
    MinX = extentBBOX[1]
    MaxY = extentBBOX[2]
    MaxX = extentBBOX[3]
    
    #Starting points. Start with MaxY and MinX so the tiles start building from the Northwest corner.
    Lat = MaxY
    Lon = MinX
    Lats = []
    Lons = []
    
    #Each bbox will be 2x2 by lines of latitude.
    while Lat > MinY:
        Lats.append(Lat)
        Lat = Lat-2
    Lats.append(MinY)

    while Lon < MaxX:
        Lons.append(Lon)
        Lon = Lon+2
    Lons.append(MaxX)
    
    #Loop through the rows (Lons) and columns (Lats) to create a dictionary of bboxes and tiles.
    tiles = {}
    for i in range(len(Lats)-1):
        for j in range(len(Lons)-1):
            
            #add keys and values
            tiles['_r' + str(j) + '_c' + str(i)] = str(Lats[j+1])+ ',' + str(Lons[i]) + ',' + str(Lats[j]) + ',' + str(Lons[i+1])

    return tiles

tiles = BBOX_creation(extentBBOX)
print(tiles)

{'_r0_c0': '47.3066,-113.291,49.3066,-111.291', '_r1_c0': '45.3066,-113.291,47.3066,-111.291', '_r2_c0': '43.3066,-113.291,45.3066,-111.291', '_r3_c0': '41.3066,-113.291,43.3066,-111.291', '_r4_c0': '39.3066,-113.291,41.3066,-111.291', '_r5_c0': '37.3066,-113.291,39.3066,-111.291', '_r6_c0': '36.4746,-113.291,37.3066,-111.291', '_r0_c1': '47.3066,-111.291,49.3066,-109.291', '_r1_c1': '45.3066,-111.291,47.3066,-109.291', '_r2_c1': '43.3066,-111.291,45.3066,-109.291', '_r3_c1': '41.3066,-111.291,43.3066,-109.291', '_r4_c1': '39.3066,-111.291,41.3066,-109.291', '_r5_c1': '37.3066,-111.291,39.3066,-109.291', '_r6_c1': '36.4746,-111.291,37.3066,-109.291', '_r0_c2': '47.3066,-109.291,49.3066,-107.291', '_r1_c2': '45.3066,-109.291,47.3066,-107.291', '_r2_c2': '43.3066,-109.291,45.3066,-107.291', '_r3_c2': '41.3066,-109.291,43.3066,-107.291', '_r4_c2': '39.3066,-109.291,41.3066,-107.291', '_r5_c2': '37.3066,-109.291,39.3066,-107.291', '_r6_c2': '36.4746,-109.291,37.3066,-107.291', '_r0_c3': '4

#### Switch back to MaxarCatalogMosaicSeamlines and loop through the bboxes to download them.

In [6]:
typename = "MaxarCatalogMosaicSeamlines"
outputFolder = "C:\\Users\\wi008132\\Downloads\\"
fileNames = 'example'

#Use the items method to loop through dictionary.
for key, value in tiles.items():
    sw_ogc.search(bbox=value, typename=typename, shapefile=True, filter = filter, download_path = outputFolder + fileNames + key + '.zip')
    print(key + ' downloaded')
    #uncomment these lines if you want to see how many features are included with each shp
    #search = sw_ogc.search(bbox=bboxes[0][i], typename=typename, filter = filter)
    #print(len(search))

_r0_c0 downloaded
_r1_c0 downloaded
_r2_c0 downloaded
_r3_c0 downloaded
_r4_c0 downloaded
_r5_c0 downloaded
_r6_c0 downloaded
_r0_c1 downloaded
_r1_c1 downloaded
_r2_c1 downloaded
_r3_c1 downloaded
_r4_c1 downloaded
_r5_c1 downloaded
_r6_c1 downloaded
_r0_c2 downloaded
_r1_c2 downloaded
_r2_c2 downloaded
_r3_c2 downloaded
_r4_c2 downloaded
_r5_c2 downloaded
_r6_c2 downloaded
_r0_c3 downloaded
_r1_c3 downloaded
_r2_c3 downloaded
_r3_c3 downloaded
_r4_c3 downloaded
_r5_c3 downloaded
_r6_c3 downloaded
_r0_c4 downloaded
_r1_c4 downloaded
_r2_c4 downloaded
_r3_c4 downloaded
_r4_c4 downloaded
_r5_c4 downloaded
_r6_c4 downloaded
_r0_c5 downloaded
_r1_c5 downloaded
_r2_c5 downloaded
_r3_c5 downloaded
_r4_c5 downloaded
_r5_c5 downloaded
_r6_c5 downloaded
_r0_c6 downloaded
_r1_c6 downloaded
_r2_c6 downloaded
_r3_c6 downloaded
_r4_c6 downloaded
_r5_c6 downloaded
_r6_c6 downloaded
